In [8]:
import os
os.environ["NVIDIA_API_KEY"] = "[YOUR_API_KEY]"

### Environment Setup

In [2]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

## Task 1: Pull In The Document Retrieval Index

In [3]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get("Title", 'unknoun')}"
        f"\n\nSummary: {doc.metadata.get("Summary", 'unknoun')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks.")
pprint("Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

Constructed aggregate docstore with 792 chunks.

Sample Chunk:

Paper: The Endogenous Constraint: Hysteresis, Stagflation, and the Structural Inhibition of Monetary Velocity in the Bitcoin Network (2016-2025)

Summary: Bitcoin operates as a macroeconomic paradox: it combines a strictly predetermined, inelastic monetary issuance schedule with a stochastic, highly elastic demand for scarce block space. This paper empirically validates the Endogenous Constraint Hypothesis, positing that protocol-level throughput limits generate a non-linear negative feedback loop between network friction and base-layer monetary velocity. Using a verified Transaction Cost Index (TCI) derived from Blockchain.com on-chain data and Hansen's (2000) threshold regression, we identify a definitive structural break at the 90th percentile of friction (TCI ~ 1.63). The analysis reveals a bifurcation in network utility: while the network exhibits robust velocity growth of +15.44% during normal regimes, this collapses to +6.06% during shock regimes, yielding a statistically signif

## Task 2:  Pull In The RAG Chain

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_title = getattr(doc, "metadata", {}).get("Title", title)
        if doc_title: out_str += f"[Quote from {doc_title}]"
        out_str += getattr(doc, "page_content", str(doc)) + "\n"

    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents"
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = chat_prompt | llm
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

I've got something interesting for you. Did you know that the Bitcoin halving, which is a reduction in the block reward for miners, has a significant impact on the price of Bitcoin? According to a study, "Estimating the Impact of the Bitcoin Halving on Its Price Using Synthetic Control" (2023), the relative search interest for 'Bitcoin Halving' has grown consistently before each halving event, with a notable spike in 2023, about 24 weeks before the 2024 halving date.

In fact, the study found that the wallet value trajectory began diverging in March 2024, about 50 days prior to the Bitcoin halving. This suggests that the market may be reacting to the upcoming halving event, which could have implications for the price of Bitcoin.

Isn't that interesting?

## Step 3: Generating Synthetic Question-Answer Pairs

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.

In [5]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_chain = ChatPromptTemplate.from_messages([("system", "{system}"), ("user", "INPUT: {input}")])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair:"
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_chain | llm).invoke({"system": sys_msg, "input": usr_msg})
    synth_questions += [qa_pair.split("\n\n")[0]]
    synth_answers += [qa_pair.split("\n\n")[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does the structure of the Bitcoin network contribute to its persistently unequal and increasingly 
core-periphery structure, as observed in its temporal evolution?

Answer: The structure of the Bitcoin network, as analyzed through its connected components and directed 
assortativity, reveals a persistently unequal and increasingly core-periphery structure. This is characterized by 
the existence of a few large-scale connected components that function as the critical backbone of the system, 
indicating a concentration of structural influence within the distribution of users. The network's 
direction-sensitive centrality measures, such as PageRank and HITS, further reveal this unequal structure, with 
high-degree nodes preferentially connecting with low-degree nodes, reflecting a hierarchical flow pattern in which 
central hubs interact asymmetrically with peripheral users.

QA Pair 2

Question: How might the volatility of Bitcoin prices impact the design of a framework like the Treasury Proof 
Ledger (TPL), which aims to provide transparency and accountability for Bitcoin treasuries?

Answer: The extreme volatility of Bitcoin prices, as observed in the study using classical time series models, 
could influence the design of TPL by necessitating the consideration of potential price fluctuations when defining 
exposure soundness, policy completeness, and other security notions. This might involve incorporating mechanisms to
account for variations in Bitcoin's value over time, ensuring that the TPL framework remains effective in providing
accurate and reliable proof-of-reserves and proof-of-transit receipts despite market volatility.

QA Pair 3

Question: How does the Bayesian linear model used in the study account for the relationship between informational 
value dimensions and next-day returns in Bitcoin?

Answer: The Bayesian linear model, operationalizing four informational value dimensions: Store-of-Value (SOV), 
Autonomy (AUT), Social-Signal Value (SSV), and Hedonic-Sentiment Value (HSV), was estimated using daily data from 
2022 to 2025. The model's residual variance is captured by a constant term, and the coefficient values follow a 
normal distribution with a mean and a standard deviation. The model's logical network is visualized in Figure 3, 
and weakly informative priors were applied to ensure stable estimation and reduce the risk of overfitting.

## Step 4: Answer The Synthetic Questions

- Use the RAG agent to generate its own answer.

In [6]:
## Generate some synthetic answers to the questions above.
rag_answers = []
for i, q in enumerate(synth_questions):
    ## Compute the RAG Answer
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", end="\n")
    pprint(rag_answer)

QA Pair 1 Question: How does the structure of the Bitcoin network contribute to its persistently unequal and 
increasingly core-periphery structure, as observed in its temporal evolution? 

Based on the study "Concentration Within Distribution: Unmasking Bitcoin's Structural Centralization Through 
Network Science," it appears that the structure of the Bitcoin network contributes to its persistently unequal and 
increasingly core-periphery structure in several ways.

Firstly, the study found that the Bitcoin User Network (BUN) evolves toward an asymmetric mesoscopic structure, 
where a few large-scale connected components function as the critical backbone of the system. This is evident in 
the growth of the network, where the total number of nodes increases steadily, while the subgraph of active nodes 
with a degree greater than 2 exhibits slower and more stable growth (Figure 10).

Secondly, the study used direction-sensitive centrality measures based on PageRank and HITS to analyze the 
distribution of structural influence within the BUN. These measures revealed a clear pattern of concentration 
within distribution, where an extremely small subset of nodes concentrates both incoming and outgoing structural 
roles (Figure 12).

Lastly, the study found that this structure produces a persistent and asymmetric core-periphery topology, 
characterized by a high influence inequality and directional concentration. This is evident in the evolution of 
Newman's four directed assortativity coefficients (rout,out, rout,in, rin,out, rin,in) for the BUN from 2011 up to 
late 2025 (Figure 9).

Overall, these findings suggest that the structure of the Bitcoin network contributes to its persistently unequal 
and increasingly core-periphery structure by creating a few large-scale connected components that function as the 
critical backbone of the system, leading to a high influence inequality and directional concentration.

QA Pair 2 Question: How might the volatility of Bitcoin prices impact the design of a framework like the Treasury 
Proof Ledger (TPL), which aims to provide transparency and accountability for Bitcoin treasuries? 

When it comes to the volatility of Bitcoin prices, it's essential to consider its impact on the design of a 
framework like the Treasury Proof Ledger (TPL). According to the document, "Meister models the interaction between 
treasury Bitcoin holdings, capital structure and market dynamics, and shows how large positions can create 
fragility and option-like payoffs" [1]. This suggests that the volatility of Bitcoin prices can significantly 
affect the balance sheet and market dynamics of a company's treasury.

In the context of TPL, this volatility could impact the framework's design in a few ways. Firstly, it might be 
necessary to incorporate more advanced modeling techniques to account for the potential risks and fragilities 
associated with large Bitcoin positions. This could involve developing more sophisticated algorithms for predicting
price movements and identifying potential vulnerabilities.

Secondly, the volatility of Bitcoin prices could also impact the way TPL interacts with broader governance 
processes. As the document notes, "modelling how strategic actors might shape transaction patterns or reporting 
schedules, and how TPL interacts with broader governance processes, seems essential for understanding which classes
of fraud become significantly more detectable in practice" [1]. This suggests that the volatility of Bitcoin prices
could influence the way TPL is integrated into existing governance frameworks and regulatory structures.

Lastly, the volatility of Bitcoin prices might also impact the way TPL is used to provide transparency and 
accountability for Bitcoin treasuries. For example, the document notes that TPL is designed to provide "replayable 
audit trails" for Bitcoin treasuries [2]. However, if Bitcoin prices are highly volatile, it may be more 
challenging to establish a clear and consistent audit trail, as the value of the treasuries may fluctuate rapidly.

Overall, the volatility of Bitcoin prices presents both opportunities and challenges for the design of a framework 
like TPL. By incorporating more advanced modeling techniques and considering the potential risks and fragilities 
associated with large Bitcoin positions, TPL can provide more accurate and reliable transparency and accountability
for Bitcoin treasuries.

QA Pair 3 Question: How does the Bayesian linear model used in the study account for the relationship between 
informational value dimensions and next-day returns in Bitcoin? 

The study uses a Bayesian linear model to assess how daily shifts in perceived value dimensions affect short-run 
Bitcoin returns. According to the study, the model was estimated using daily data from 2022 to 2025, 
operationalizing four informational value dimensions: Store-of-Value (SOV), Autonomy (AUT), Social-Signal Value 
(SSV), and Hedonic-Sentiment Value (HSV) [1].

The Bayesian linear model takes into account the lagged changes in these value dimensions and their association 
with next-day Bitcoin returns. The results indicate that only SSV exerts a highly credible positive effect on 
next-day returns, highlighting the dominant role of high-entropy social information in short-term pricing dynamics.
On the other hand, SOV and AUT show moderately reliable positive associations, reflecting their roles as 
low-entropy structural anchors of long-term value [1].

The model was estimated using Markov Chain Monte Carlo (MCMC) simulation, and the posterior estimation was 
performed under weakly informative priors to promote data-driven inference while maintaining parameter stability. 
The model fit was evaluated, and the results demonstrate a strong correspondence between the model and the observed
data [1].

In summary, the Bayesian linear model used in the study accounts for the relationship between informational value 
dimensions and next-day returns in Bitcoin by estimating the lagged changes in these value dimensions and their 
association with next-day returns.

[1] Bayesian probabilistic exploration of Bitcoin informational quanta and interactions under the GITT-VT paradigm.

## Step 5: Implement A Human Preference Metric

- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

In [7]:
## Adapt this prompt for whichever LLM you're actually interested in using.
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION:
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION:
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\nAnswer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({"qa_trio": qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does the structure of the Bitcoin network contribute to its persistently unequal and 
increasingly core-periphery structure, as observed in its temporal evolution?

Synth Answer: Answer: The structure of the Bitcoin network, as analyzed through its connected components and 
directed assortativity, reveals a persistently unequal and increasingly core-periphery structure. This is 
characterized by the existence of a few large-scale connected components that function as the critical backbone of 
the system, indicating a concentration of structural influence within the distribution of users. The network's 
direction-sensitive centrality measures, such as PageRank and HITS, further reveal this unequal structure, with 
high-degree nodes preferentially connecting with low-degree nodes, reflecting a hierarchical flow pattern in which 
central hubs interact asymmetrically with peripheral users.

RAG Answer: Based on the study "Concentration Within Distribution: Unmasking Bitcoin's Structural Centralization 
Through Network Science," it appears that the structure of the Bitcoin network contributes to its persistently 
unequal and increasingly core-periphery structure in several ways.

Firstly, the study found that the Bitcoin User Network (BUN) evolves toward an asymmetric mesoscopic structure, 
where a few large-scale connected components function as the critical backbone of the system. This is evident in 
the growth of the network, where the total number of nodes increases steadily, while the subgraph of active nodes 
with a degree greater than 2 exhibits slower and more stable growth (Figure 10).

Secondly, the study used direction-sensitive centrality measures based on PageRank and HITS to analyze the 
distribution of structural influence within the BUN. These measures revealed a clear pattern of concentration 
within distribution, where an extremely small subset of nodes concentrates both incoming and outgoing structural 
roles (Figure 12).

Lastly, the study found that this structure produces a persistent and asymmetric core-periphery topology, 
characterized by a high influence inequality and directional concentration. This is evident in the evolution of 
Newman's four directed assortativity coefficients (rout,out, rout,in, rin,out, rin,in) for the BUN from 2011 up to 
late 2025 (Figure 9).

Overall, these findings suggest that the structure of the Bitcoin network contributes to its persistently unequal 
and increasingly core-periphery structure by creating a few large-scale connected components that function as the 
critical backbone of the system, leading to a high influence inequality and directional concentration.

Synth Evaluation: [Score: 1] Justification: 

Answer 2 is actually a rephrased version of Answer 1 with additional information from a study. However, the core 
idea remains the same, and it doesn't introduce any inconsistencies with the ground truth answer. The second answer
provides more detailed evidence from the study to support the claim, making it a better and more informative answer
than the first. Therefore, it scores 1, indicating that the second answer is better than the first and does not 
introduce any inconsistencies.

Set 2

Question: Question: How might the volatility of Bitcoin prices impact the design of a framework like the Treasury 
Proof Ledger (TPL), which aims to provide transparency and accountability for Bitcoin treasuries?

...

Synth Evaluation: [Score] 1.0 
Justification: 
The second answer is not only better than the first but also does not introduce any inconsistencies. It provides 
more detailed information and insights on how the volatility of Bitcoin prices can impact the design of the 
Treasury Proof Ledger (TPL) framework. It also references a document that supports its claims, which adds 
credibility to the answer. The second answer also presents a more comprehensive analysis of the potential risks and
fragilities associated with large Bitcoin positions and how TPL can be designed to mitigate these risks. Overall, 
the second answer provides a more thorough and well-supported response to the question, making it a more accurate 
and reliable answer.

Set 3

Question: Question: How does the Bayesian linear model used in the study account for the relationship between 
informational value dimensions and next-day returns in Bitcoin?

Synth Answer: Answer: The Bayesian linear model, operationalizing four informational value dimensions: 
Store-of-Value (SOV), Autonomy (AUT), Social-Signal Value (SSV), and Hedonic-Sentiment Value (HSV), was estimated 
using daily data from 2022 to 2025. The model's residual variance is captured by a constant term, and the 
coefficient values follow a normal distribution with a mean and a standard deviation. The model's logical network 
is visualized in Figure 3, and weakly informative priors were applied to ensure stable estimation and reduce the 
risk of overfitting.

RAG Answer: The study uses a Bayesian linear model to assess how daily shifts in perceived value dimensions affect 
short-run Bitcoin returns. According to the study, the model was estimated using daily data from 2022 to 2025, 
operationalizing four informational value dimensions: Store-of-Value (SOV), Autonomy (AUT), Social-Signal Value 
(SSV), and Hedonic-Sentiment Value (HSV) [1].

The Bayesian linear model takes into account the lagged changes in these value dimensions and their association 
with next-day Bitcoin returns. The results indicate that only SSV exerts a highly credible positive effect on 
next-day returns, highlighting the dominant role of high-entropy social information in short-term pricing dynamics.
On the other hand, SOV and AUT show moderately reliable positive associations, reflecting their roles as 
low-entropy structural anchors of long-term value [1].

The model was estimated using Markov Chain Monte Carlo (MCMC) simulation, and the posterior estimation was 
performed under weakly informative priors to promote data-driven inference while maintaining parameter stability. 
The model fit was evaluated, and the results demonstrate a strong correspondence between the model and the observed
data [1].

In summary, the Bayesian linear model used in the study accounts for the relationship between informational value 
dimensions and next-day returns in Bitcoin by estimating the lagged changes in these value dimensions and their 
association with next-day returns.

[1] Bayesian probabilistic exploration of Bitcoin informational quanta and interactions under the GITT-VT paradigm.

Synth Evaluation: [Score: 2] Justification:
The second answer is better than the first and does not introduce any inconsistencies. 

The second answer provides more specific and detailed information about the study, including the results of the 
Bayesian linear model. It explains how the model accounts for the relationship between informational value 
dimensions and next-day returns in Bitcoin, and it provides more information about the statistical methods used, 
such as Markov Chain Monte Carlo (MCMC) simulation and weakly informative priors.

The second answer also provides more insight into the specific relationships between the informational value 
dimensions and next-day returns, including the dominant role of high-entropy social information in short-term 
pricing dynamics. This information is not present in the first answer, which only mentions the operationalization 
of the four informational value dimensions.

Overall, the second answer provides more detail and insight into the study, and it does not introduce any 
inconsistencies with the information provided in the first answer.